In [ ]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from colabgymrender.recorder import Recorder

# Load The Enviorment

In [ ]:
enviorment_name = 'CartPole-v0'
env = gym.make(enviorment_name)
env = Recorder(env, './video')

In [ ]:
# Test the enviorment
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

# Understanding the enviorment

In [ ]:
env.action_space

In [ ]:
env.action_space.sample()

In [ ]:
env.observation_space

In [ ]:
env.observation_space.sample()

Observation (Box(4)):
    0 - Cart position: -4.8 to 4.8
    1 - Cart velocity: -inf to inf
    2 - Pole angle: -0.418 rad to 0.418 rad
    3 - Pole angular velocity: -inf to inf

Action(Discret(2)):
    0 - Left
    1 - Right

# Traing RL Model

In [ ]:
log_path = os.path.join('Training', 'Logs')
log_path

In [ ]:
env = gym.make(enviorment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)

# Save and Reload the Model

In [ ]:
# save the model
PPO_Path = os.path.join('Training', 'saved_models', 'PPO_model_CartPole')
model.save(PPO_Path)

In [ ]:
# Load the model
del model
model = PPO.load(PPO_Path, env=env)

# Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

# Testing the Model   

In [ ]:
# Test the enviorment
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

# Viewing Logs on TensorBoard 

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_1')
training_log_path

In [ ]:
!tensorboard --logdir={training_log_path}

# Adding a Callback on the training stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join('Training', 'saved_models')
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                            callback_on_new_best=stop_callback, 
                            eval_freq=10000, 
                            best_model_save_path=save_path, 
                            verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

Using an atlernate algotirthm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
DQN_path = os.path.join('Training', 'saved_models', 'DQN_model_CartPole')
model.save(DQN_path)